In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


# Loading the VGG16 Model

In [2]:
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 160
img_cols = 160 

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# freeze all layer except top 4

In [3]:
# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


# make function that return fully-connected head

In [4]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [5]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 160, 160, 3) dtype=float32>

In [6]:
model.layers

# add our fully-connected head abck onto vgg16

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 3

FC_Head = addTopModel(model, num_classes)

modelnew = Model(inputs=model.input, outputs=FC_Head)

print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 160, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 160, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 80, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 80, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 80, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 40, 128)       0   

# loading face database

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/my/linuxworld_programs/mlops/task3_face_recog/face_sample/train/'
validation_data_dir = 'C:/my/linuxworld_programs/mlops/task3_face_recog/face_sample/validation/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 492 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


# training our top layer

In [9]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("face_recog_VGG16.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 5
batch_size = 16

history = modelnew.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/5
74/74 [==============================] - 119s 2s/step - loss: 0.9340 - accuracy: 0.8285 - val_loss: 8.5930e-04 - val_accuracy: 0.9900

Epoch 00001: val_loss improved from inf to 0.00086, saving model to face_recog_VGG16.h5
Epoch 2/5
74/74 [==============================] - 114s 2s/step - loss: 0.1826 - accuracy: 0.9458 - val_loss: 1.7849 - val_accuracy: 0.9300

Epoch 00002: val_loss did not improve from 0.00086
Epoch 3/5
74/74 [==============================] - 109s 1s/step - loss: 0.0506 - accuracy: 0.9812 - val_loss: 0.0044 - val_accuracy: 1.0000

Epoch 00003: val_loss did not improve from 0.00086
Epoch 4/5
74/74 [==============================] - 117s 2s/step - loss: 0.0608 - accuracy: 0.9821 - val_loss: 5.9722e-06 - val_accuracy: 0.9900

Epoch 00004: val_loss improved from 0.00086 to 0.00001, saving model to face_recog_VGG16.h5
Epoch 5/5
74/74 [==============================] - 123s 2s/step - loss: 0.0225 - accuracy: 0.9915 - val_loss: 0.1586 - val_accuracy: 0.9900

Epoch

In [10]:
modelnew.save("face_recog_VGG16.h5")